In [1]:
from FlagEmbedding import FlagModel
import numpy as np
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pprint import pprint as pp
import time
import umap
import os
import random
import time
from contextlib import contextmanager
import torch
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, load_from_disk
from datasets import load_dataset, concatenate_datasets, load_from_disk
import pandas as pd
import datasets
from datasets import Dataset
from pprint import pprint as pp
from datasets import Dataset
from sklearn.cluster import KMeans
from tqdm import tqdm
import torch
import heapq
import sys
from vllm import LLM

sys.path.append("/mnt/bn/data-tns-live-llm/leon/Cherry_LLM")
from niid_data.utils import process_sft_dataset

WARNING 08-28 13:44:45 cuda.py:22] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead. See https://pypi.org/project/pynvml for more information.


In [2]:
code_data = load_dataset("sahil2801/CodeAlpaca-20k")["train"]
fin_data = load_dataset("FinGPT/fingpt-sentiment-train")["train"]
med_data = load_dataset("medalpaca/medical_meadow_medical_flashcards")["train"]
general_data = load_dataset("tatsu-lab/alpaca")["train"]
math_data = load_dataset("TIGER-Lab/MathInstruct")["train"]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/262039 [00:00<?, ? examples/s]

In [3]:
import random
random.seed(42)
client_data = process_sft_dataset("sahil2801/CodeAlpaca-20k",code_data)
iid_idxs = random.sample(range(len(client_data)), 1000)
base_data = client_data.select(iid_idxs)
clients_data = []
for i in range(10):
    clients_data.append(base_data.shard(10,i))

>> ===== After processing, Dataset sahil2801/CodeAlpaca-20k has 20022 examples. =====


In [4]:
import requests
import json

def get_gpt(prompt, format):
    data = {
        "model": "gpt-3.5-turbo-0125",
        "messages":  [
            {"role": "system", "content": f"You are a helpful assistant designed to output JSON. The format is: {format}"},
            {"role": "user", "content": prompt}
        ],
        "temperature": 1,
        "top_p": 1,
        "n": 1,
        "stream": False,
        "stop": None,
        "max_tokens": 512,
        "presence_penalty": 0,
        "frequency_penalty": 0,
        "user": None,
        "response_format": {"type": "json_object"}
    }
    headers = {'Content-Type': 'application/json', 'Caller': 'leon.kepler'}
    data = {k: v for k, v in data.items() if v is not None}
    data = json.dumps(data)
    url = f"https://swzkkd0h.us-east-fn.bytedance.net/gpt/openapi/online/v2/crawl"
    response = requests.post(url, data=data, headers=headers)
    return response.json()["choices"][0]["message"]["content"]

# unsloth 本地运行

In [2]:
from unsloth import FastLanguageModel
from peft import PeftModelForCausalLM
from transformers import LlamaTokenizer
model: PeftModelForCausalLM
tokenizer: LlamaTokenizer
model, tokenizer = FastLanguageModel.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/llama-3-8b-Instruct-bnb-4bit",dtype = torch.bfloat16,
    load_in_4bit = True)
FastLanguageModel.for_inference(model)
print("Done")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
[2024-08-26 17:45:56,359] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/tiger/.local/lib/python3.9/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/tiger/.local/lib/python3.9/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA A100-SXM4-80GB. Max memory: 79.347 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Done


In [13]:
def generate_one_completion(instance, prefix_function):
    inputs = tokenizer(instance, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    generate_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=1024, repetition_penalty=1.1, do_sample=True, prefix_allowed_tokens_fn=prefix_function, min_new_tokens=50)
    outputs = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return outputs

In [ ]:
from pydantic import BaseModel
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn
from transformers import pipeline

class AnswerFormat(BaseModel):
    Instruction: str

# Create a transformers pipeline
# hf_pipeline = pipeline('text-generation', model='TheBloke/Llama-2-7b-Chat-GPTQ', device_map='auto')
prompt = f"""You are asked to come up with instructions. These instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions. Don't repeat instructions in examples. Here are some examples: 
Instruction 1: Write a for loop in JavaScript that prints numbers from 0 to 5. 
Instruction 2: Compare two objects and return 0 if they are equal, -1 if the first is less than the second, and 1 if the first is greater than the second. object1 = 7 object2 = 8 
Instruction 3: Develop a code to sort a list in ascending or descending order. givenList: [5, -3, 2, 4, 0], sortOrder: "ascending" 
Instruction 4: Insert a node at the beginning of a linked list. [10 -> 15 -> 20 -> 25] 
Provide a new instruction in the following json schema: {AnswerFormat.schema_json()} :\n"""

# Create a character level parser and build a transformers prefix function from it
parser = JsonSchemaParser(AnswerFormat.schema())
prefix_function = build_transformers_prefix_allowed_tokens_fn(tokenizer, parser)

# Call the pipeline with the prefix function
output_dict = generate_one_completion(prompt, prefix_function)
# Extract the results
result = output_dict[len(prompt):]
print(result)

# 尝试 vllm json mode

In [4]:
instruction_schema = """{
    "title": "Instruction Schema",
    "type": "object",
    "properties": {
        "Instruction": {
            "title": "instruction",
            "type": "string"
        }
    },
    "required": ["Instruction"]
}"""

response_schema = """{
    "title": "Response Schema",
    "type": "object",
    "properties": {
        "Response": {
            "title": "response",
            "type": "string"
        }
    },
    "required": ["Response"]
}"""

In [11]:
from openai import OpenAI
import json
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="damn"
)

def get_vllm(prompt, schema):
    num_tokens=0
    if "Response" in schema: 
        num_tokens=50

    cnt=0
    while cnt!=2:
        cnt+=1
        output = None
        completion = client.chat.completions.create(
            model="Meta-Llama-3-8B-Instruct/",
            messages=[
                {"role": "system", "content": f"You are a helpful assistant designed to output JSON."},
                {"role": "user", "content": prompt},
            ],
            extra_body={
                "repetition_penalty": 1.1,
                "min_tokens": num_tokens,
                "guided_json": schema
            }
        )
        try:
            output = json.loads(completion.choices[0].message.content)
            break
        except Exception as e:
            print(f"gpt error {e}")
            print(f"gpt output {completion}")
    return output

# 生成response

In [4]:
instructions = []
response_examples = []
for i in range(100):
    instructions.append(random.sample(clients_data[0]["instruction"],4))
    response_examples.append(clients_data[0].select(random.sample(range(len(clients_data[0])),2)))

In [44]:
response_examples[0]["instruction"]
response_examples[0]["response"]

['# Initialize the first two numbers\na = 0\nb = 1\n\n# Print the first 10 Fibonacci numbers\nfor i in range(10):\n    # Swap the values\n    a, b = b, a + b\n    # Print the current Fibonacci number\n    print(b)\n# Reverse the list\nfor _ in range(9, 0, -1):\n    print(_)',
 'UPDATE customers \nSET age = 30\nWHERE customer_id = 24;']

In [45]:
for data in response_examples[0]:
    print(data)

{'instruction': 'Write an algorithm that prints the first 10 Fibonacci numbers in descending order.', 'response': '# Initialize the first two numbers\na = 0\nb = 1\n\n# Print the first 10 Fibonacci numbers\nfor i in range(10):\n    # Swap the values\n    a, b = b, a + b\n    # Print the current Fibonacci number\n    print(b)\n# Reverse the list\nfor _ in range(9, 0, -1):\n    print(_)'}
{'instruction': 'Write a SQL statement to update the age of a customer with a specific ID. Customer_id = 24, Age = 30', 'response': 'UPDATE customers \nSET age = 30\nWHERE customer_id = 24;'}


In [12]:
import evaluate
rouge = evaluate.load('rouge')
def compute_sim(instruction):
    return rouge.compute(predictions=[instruction], references=[clients_data[0]["instruction"]],rouge_types=['rougeL'])["rougeL"]

In [5]:
instruction_prompt = """You are asked to come up with instructions. These instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions. Don't repeat instructions in examples. Here are some examples: Instruction 1: {} Instruction 2: {} Instruction 3: {} Instruction 4: {} Provide a new instruction below:"""

instruction_format = "{Instruction: instruction}"

In [8]:
def generate_instruction(prompt, instruction_format=instruction_format):
    cnt=0
    while cnt!=2:
        cnt+=1
        instruction = get_vllm(prompt, instruction_schema)
        # try:
        #     # instruction = json.loads(get_gpt(prompt,instruction_format))["Instruction"]
        #     instruction = get_vllm(prompt, instruction_schema)["Instruction"]
        # except:
        #     print("generate instruction error")
        #     continue
        # if compute_sim(instruction)>0.7: 
        #     print(instruction)
        #     continue
        break
    return instruction

In [9]:
response_format="{Response: response}"

def generate_response(prompt, response_format=response_format):
    cnt=0
    output = None
    while cnt!=2:
        cnt+=1
        # output = get_gpt(prompt,response_format)
        output = get_vllm(prompt,response_schema)
        break
        # try:
        #     # output = json.loads(output)["Response"]
        #     output = output["Response"]
        #     break
        # except:
        #     print(output)
        #     # time.sleep(1)
    return output

In [12]:
tmp = instruction_prompt.format(*instructions[0])
generate_instruction(tmp,instruction_format)

{'Instruction': 'Write a function in Python to count the number of vowels in a given string.'}

: 

In [100]:
# response_format = """You are asked to generate the response according to the instruction. Here are some examples:
# Example 1: 
#     Instruction: {}
#     Response: {}
# Example 2: 
#     Instruction: {}
#     Response: {}

# Generate the response of the following instruction: {}
# """
response_format = """Instruction: {}
Response: {}
Instruction: {}
Response: {}
Instruction: {}
"""

In [24]:
def format_response_prompt(idx, instruction):
    response_seqs = []
    for data in response_examples[idx]:
        response_seqs.extend([data["instruction"],data["response"]])
    response_seqs.append(instruction)
    prompt = response_format.format(*response_seqs)
    return prompt

In [95]:
response_seqs = []
for data in response_examples[0]:
    response_seqs.extend([data["instruction"],data["response"]])
print(response_seqs)

['Write an algorithm that prints the first 10 Fibonacci numbers in descending order.', '# Initialize the first two numbers\na = 0\nb = 1\n\n# Print the first 10 Fibonacci numbers\nfor i in range(10):\n    # Swap the values\n    a, b = b, a + b\n    # Print the current Fibonacci number\n    print(b)\n# Reverse the list\nfor _ in range(9, 0, -1):\n    print(_)', 'Write a SQL statement to update the age of a customer with a specific ID. Customer_id = 24, Age = 30', 'UPDATE customers \nSET age = 30\nWHERE customer_id = 24;']


In [96]:
instruction = "Write a Python function that takes a string as input and returns the string with all vowels removed."
response_seqs.append(instruction)
tmp_prompt = response_format.format(*response_seqs)
tmp_prompt

'\nExample 1: \n    Instruction: Write an algorithm that prints the first 10 Fibonacci numbers in descending order.\n    Response: # Initialize the first two numbers\na = 0\nb = 1\n\n# Print the first 10 Fibonacci numbers\nfor i in range(10):\n    # Swap the values\n    a, b = b, a + b\n    # Print the current Fibonacci number\n    print(b)\n# Reverse the list\nfor _ in range(9, 0, -1):\n    print(_)\nExample 2: \n    Instruction: Write a SQL statement to update the age of a customer with a specific ID. Customer_id = 24, Age = 30\n    Response: UPDATE customers \nSET age = 30\nWHERE customer_id = 24;\n\nGenerate the response of this instruction, Instruction: Write a Python function that takes a string as input and returns the string with all vowels removed.\n'

In [94]:
generate_response(tmp_prompt)

"def remove_vowels(input_string):\\\\n    vowels = 'aeiouAEIOU'\\\\n    output_string = ''\\\\n    for char in input_string:\\\\n        if char not in vowels:\\\\n            output_string += char\\n    return output_string"

In [19]:
gen_instructions = []
gen_responses = []
for i in tqdm(range(50)):
    tmp = instruction_prompt.format(*instructions[i])
    instruction = generate_instruction(tmp,instruction_format)
    prompt = format_response_prompt(i,instruction)
    response = generate_response(prompt)
    gen_instructions.append(instruction)
    gen_responses.append(response)

100%|██████████| 50/50 [03:01<00:00,  3.62s/it]


# vllm batch input

In [ ]:
formatted_instruction_prompt = [instruction_prompt.format(*instruction) for instruction in instructions]
formatted_instruction_prompt[:5]

In [ ]:
gen_instructions = generate_instruction(formatted_instruction_prompt)

In [20]:
df = pd.DataFrame({
    "instruction": gen_instructions,
    "response": gen_responses,
})
df.to_csv(f"./code.csv")

# 月之暗面！

In [1]:
import json
 
from openai import OpenAI
 
client = OpenAI(
    api_key="sk-Nl2yxo0Ic9ZJ0zFf3qWdypuoQ6Nd2xnsCp7wfojlW9OyOYmq", # 在这里将 MOONSHOT_API_KEY 替换为你从 Kimi 开放平台申请的 API Key
    base_url="https://api.moonshot.cn/v1",
)

# {
#     "response": "text",
# }
prompt = """You are asked to come up with instructions. These instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions. Don't repeat instructions in examples. Here are some examples: 
Instruction 1: Write a for loop in JavaScript that prints numbers from 0 to 5. 
Instruction 2: Compare two objects and return 0 if they are equal, -1 if the first is less than the second, and 1 if the first is greater than the second. object1 = 7 object2 = 8 
Instruction 3: Develop a code to sort a list in ascending or descending order. givenList: [5, -3, 2, 4, 0], sortOrder: "ascending" 
Instruction 4: Insert a node at the beginning of a linked list. [10 -> 15 -> 20 -> 25] 
Provide a new instruction below."""

system_prompt = """
You are the intelligent customer service of Dark Side of the Moon (Kimi).
Please use the following JSON format to output your response:
{"Instruction": "instruction",}
"""
 
completion = client.chat.completions.create(
    model="moonshot-v1-8k",
    messages=[
        {"role": "system", "content": system_prompt}, # <-- 将附带输出格式的 system prompt 提交给 Kimi
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=512,
    frequency_penalty=1.0,
    response_format={"type": "json_object"}, # <-- 使用 response_format 参数指定输出格式为 json_object
)
 
content = json.loads(completion.choices[0].message.content)
print(content)

{'Instruction': 'Write a Python function that calculates the factorial of a non-negative integer. The function should handle input validation to ensure the provided argument is an integer and is not negative, and it should return the factorial of the number if valid or an appropriate error message otherwise.'}


In [10]:
data_concated: Dataset = processed_data[0]
random.seed(42)
iid_idxs = random.sample(range(len(data_concated)), 1000)
base_data = data_concated.select(iid_idxs)
clients_data = []
for i in range(10):
    clients_data.append(base_data.shard(10,i))

data_concated = data_concated.select(list(set(range(len(data_concated)))-set(iid_idxs)))
print(len(data_concated))

75772


In [14]:
print(clients_data[0].column_names)

['instruction', 'response', 'label']


In [16]:
from datasets import load_dataset
import json

domain = "financial"
# 定义转换函数，将数据集中的一行转换为所需的JSONL格式
def convert_to_jsonl(i,row):
    return {
        "id": f"seed_task_{i}",
        "name": row['name'] if 'name' in row else domain,  # 假设每行都有'name'列，如果没有，则使用默认值
        "instruction": row['instruction'],
        "instances": [{"input": "", "output": row['response']}],
        "is_classification": False
    }

# 打开文件用于写入
with open('output.jsonl', 'w') as f:
    # 遍历数据集中的所有行
    for i,item in enumerate(clients_data[0]):  # 假设我们处理的是训练集
        # 转换为JSONL格式
        jsonl_line = convert_to_jsonl(i,item)
        # 将JSON对象转换为JSON字符串，并写入文件
        f.write(json.dumps(jsonl_line) + '\n')

print("转换完成，文件已保存为 output.jsonl")

转换完成，文件已保存为 output.jsonl


# vllm async api

In [6]:
instructions_formatted = [instruction_prompt.format(*instruction) for instruction in instructions]

In [7]:
import json
import os
import sys
import time
import traceback
from dataclasses import dataclass, field
from typing import List, Optional, Union

import aiohttp
import huggingface_hub.constants
from tqdm.asyncio import tqdm
from transformers import (AutoTokenizer, PreTrainedTokenizer,
                          PreTrainedTokenizerFast,PreTrainedTokenizerBase)
from typing import Any, AsyncGenerator, Dict, List, Optional, Tuple
import asyncio


AIOHTTP_TIMEOUT = aiohttp.ClientTimeout(total=6 * 60 * 60)

In [8]:
async def get_request(
    input_requests: List[Tuple[str, int, int]],
    request_rate: float,
) -> AsyncGenerator[Tuple[str, int, int], None]:
    input_requests = iter(input_requests)
    for request in input_requests:
        yield request

        if request_rate == float("inf"):
            # If the request rate is infinity, then we don't need to wait.
            continue

        # Sample the request interval from the exponential distribution.
        interval = np.random.exponential(1.0 / request_rate)
        # The next request will be sent after the interval.
        await asyncio.sleep(interval)

In [9]:
@dataclass
class RequestFuncInput:
    prompt: str
    api_url: str
    model: str
    best_of: int = 1
    use_beam_search: bool = False

In [10]:
@dataclass
class RequestFuncOutput:
    generated_text: str = ""
    success: bool = False
    latency: float = 0.0
    ttft: float = 0.0  # Time to first token
    itl: List[float] = field(
        default_factory=list)  # List of inter-token latencies
    error: str = ""

In [11]:
instruction_schema = """{
    "title": "Instruction Schema",
    "type": "object",
    "properties": {
        "Instruction": {
            "title": "instruction",
            "type": "string"
        }
    },
    "required": ["Instruction"]
}"""

response_schema = """{
    "title": "Response Schema",
    "type": "object",
    "properties": {
        "Response": {
            "title": "response",
            "type": "string"
        }
    },
    "required": ["Response"]
}"""

In [107]:
def remove_prefix(text: str, prefix: str) -> str:
    if text.startswith(prefix):
        return text[len(prefix):]
    return text
    
async def async_request_openai_completions(
    request_func_input: RequestFuncInput,
    format: str,
    pbar: Optional[tqdm] = None,
) -> RequestFuncOutput:
    api_url = request_func_input.api_url
    assert api_url.endswith(
        ("completions", "profile")
    ), "OpenAI Completions API URL must end with 'completions' or 'profile'."

    async with aiohttp.ClientSession(timeout=AIOHTTP_TIMEOUT) as session:
        assert not request_func_input.use_beam_search
        payload = {
            "model": request_func_input.model,
            "prompt": json.dumps(
                [
                    # {"role": "system", "content": f"You are a helpful assistant designed to generate response to the instruction in JSON"},
                    {"role": "user", "content": f"{request_func_input.prompt}"}
                ]),
            "temperature": 0.7,
            "best_of": request_func_input.best_of,
            "max_tokens": 512,
            "min_tokens": 50,
            # "guided_json": format,
            "stream": True,
            "frequency_penalty": 1.0,
        }
        headers = {
            "Authorization": f"Bearer {os.environ.get('OPENAI_API_KEY')}"
        }

        output = RequestFuncOutput()

        generated_text = ""
        ttft = 0.0
        st = time.perf_counter()
        most_recent_timestamp = st
        try:
            async with session.post(url=api_url, json=payload,
                                    headers=headers) as response:
                if response.status == 200:
                    async for chunk_bytes in response.content:
                        chunk_bytes = chunk_bytes.strip()
                        if not chunk_bytes:
                            continue

                        chunk = remove_prefix(chunk_bytes.decode("utf-8"),
                                              "data: ")
                        if chunk == "[DONE]":
                            latency = time.perf_counter() - st
                        else:
                            data = json.loads(chunk)

                            # NOTE: Some completion API might have a last
                            # usage summary response without a token so we
                            # want to check a token was generated
                            if data["choices"][0]["text"]:
                                timestamp = time.perf_counter()
                                # First token
                                if ttft == 0.0:
                                    ttft = time.perf_counter() - st
                                    output.ttft = ttft

                                # Decoding phase
                                else:
                                    output.itl.append(timestamp -
                                                      most_recent_timestamp)

                                most_recent_timestamp = timestamp
                                generated_text += data["choices"][0]["text"]

                    output.generated_text = generated_text
                    output.success = True
                    output.latency = latency
                else:
                    output.error = response.reason or ""
                    output.success = False
        except Exception:
            output.success = False
            exc_info = sys.exc_info()
            output.error = "".join(traceback.format_exception(*exc_info))

    if pbar:
        pbar.update(1)
    return output

In [18]:
async def vllm_run(
    api_url: str,
    model_id: str,
    input_requests: List[Tuple[str, int, int]],
    best_of: int,
    use_beam_search: bool,
    request_rate: float,
    disable_tqdm: bool,
    profile: bool,
    format: str,
    ):
    pbar = None if disable_tqdm else tqdm(total=len(input_requests))
    tasks: List[asyncio.Task] = []
    async for request in get_request(input_requests, request_rate):
        prompt = request
        request_func_input = RequestFuncInput(
            model=model_id,
            prompt=prompt,
            api_url=api_url,
            best_of=best_of,
            use_beam_search=use_beam_search,
        )
        tasks.append(
            asyncio.create_task(
                async_request_openai_completions(request_func_input=request_func_input,format=format,
                                pbar=pbar)))
    outputs: List[RequestFuncOutput] = await asyncio.gather(*tasks)
    return outputs

In [ ]:
benchmark_result = asyncio.run(
    vllm_run(
        api_url=f"http://localhost:8000/v1/completions",
        model_id="NousResearch/Meta-Llama-3-8B-Instruct",
        input_requests=instructions_formatted,
        best_of=1,
        use_beam_search=False,
        request_rate=float("inf"),
        disable_tqdm=False,
        profile=False,
        format=instruction_schema
    ))

In [19]:
result = [json.loads(res.generated_text)["Instruction"] for res in benchmark_result]

In [51]:
result[:2]

['Write a function that takes a string and returns the first 5 characters of that string. If the string is less than 5 characters, return the entire string.',
 'Write a Python script to calculate the mean of the grades of students in a class. The grades are given as input by the user and should be separated by spaces.']

In [101]:
formatted_response_prompt = [format_response_prompt(i, instruction) for i,instruction in zip(range(100),result)]

In [ ]:
formatted_response_prompt[-2:]

In [108]:
benchmark_result = asyncio.run(
    vllm_run(
        api_url=f"http://localhost:8001/v1/completions",
        # model_id="allenai/open-instruct-code-alpaca-7b",
        model_id="Meta-Llama-3.1-8B-Instruct",
        input_requests=formatted_response_prompt,
        best_of=1,
        use_beam_search=False,
        request_rate=float("inf"),
        disable_tqdm=False,
        profile=False,
        format=response_schema
    ))





































100%|██████████| 100/100 [00:20<00:00,  4.79it/s]


In [109]:
res_result = [res.generated_text for res in benchmark_result]

In [113]:
res_result[3]

' [{"role": "assistant", "content": "Here is the Python code that prints binary tree elements level by level: \\n\\nclass BinaryTree: \\n    def __init__(self, root): \\n        self.root = Node(root) \\n\\n    def printLevelOrder(self): \\n        h = self.height(self.root) \\n        for i in range(1, h+1): # to handle odd number of nodes\\n            self.printGivenLevel(self.root, i)  # <--- modified function call\\n  \\\\\\ndef height(self,node):\\ n\\tif node is None:\\ n\\t\\treturn 0\\ n\\telse:\\ n\\t\\treturn max(self.height(node.left),self.height(node.right))\\ndef printGivenLevel(root , level):\\ n   if (root == None):\\ n       return\\ n   if (level == 1):\\ ndb.printf(\\"%d \\" , root.data)\\nelse if (level > 1):\\ ndb.printGivenLevel(root.left , level-1)\\ndb.printGivenLevel(root.right , level-1)\\ndef Node(data): return TreeNode(data)"}, {"role": "assistant", "content": "Here is the Python code that represents a student with attributes and a method to get student info

In [71]:
df = pd.DataFrame({
    "instruction": result,
    "response": res_result
})
df.to_csv("code.csv")